In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from mpl_toolkits.mplot3d import axes3d 
import os
from tqdm.auto import tqdm
from numba import jit,stencil
import scipy

plt.style.use('science')

# loading all of the csv's from all of the simulations with the macro measured data (total energy, ent. excess, Paccept....)
simlist = []
datafiles = []
csvfiles = []
ic = 0
for root, dirs, files in os.walk("../data/pdanneal/"):
    for file in files:
        #print(file)
        if file.endswith(".npz"):
            datafiles.append(os.path.join(root,file))
        if file.endswith(".csv"):
            csvfiles.append(os.path.join(root,file))
    for name in dirs:
        simlist.append(name)
simlist = sorted(simlist)
datafiles = sorted(datafiles)
csvfiles = sorted(csvfiles)

print(len(simlist))
print(len(datafiles))
print(len(csvfiles))
assert len(simlist) == len(datafiles) == len(csvfiles)


def load_csv(fname,verbose=True):
    if verbose:
        with open(fname) as f:
            print(f.readline().strip('\n'))
    return np.loadtxt(fname,skiprows=1,delimiter=",")
    
def get_temp_K(csvfiles):
    Ks = []
    kbts = []
    for csvfile in csvfiles:
        csv1 = load_csv(csvfile,verbose=False)
        K = float(csvfile.split("/")[3].split("_")[-2][1:])
        kbt = float(csvfile.split("/")[3].split("_")[-1][3:])
        Ks.append(K)
        kbts.append(kbt)#/Tin)
    Ks = sorted(list(set(Ks))) #list of all Ks w/ duplicates removed
    kbts = sorted(list(set(kbts))) # list of all Kbts w/ duplicates removed
    print(Ks)
    print(kbts)
    return kbts,Ks
    
%matplotlib ipympl

default_colormap = 'RdYlBu'#'coolwarm'#'Spectral'#'YlGnBu'
default_fontsize = 24

In [ ]:
datafile = "../data/pdanneal/pdanneal_K2.0_kbt0.05/pdanneal_K2.0_kbt0.05_data.npz"
data = np.load(datafile)
print(data)
nx = data['nx']
ny = data['ny']
nz = data['nz']
s = data['s']

In [ ]:
#3d quiver plot TONS OF MEMORY
fig,ax = plt.subplots(1,1,figsize=(6,6),dpi=125,subplot_kw={"projection": "3d"})

print(nx.shape)
X,Y,Z = np.meshgrid(np.arange(nx.shape[0]),np.arange(nx.shape[1]),np.arange(nx.shape[2]))
ax.quiver(X,Y,Z,nx,ny,nz,length=0.1,normalize=True)
plt.show()

In [ ]:
@stencil
def ekernel(nx,ny,nz,s):
    KK = 1.0
    e = 0.0
    # i + 1
    dott = nx[0,0,0]*nx[1,0,0] + ny[0,0,0]*ny[1,0,0] + nz[0,0,0]*nz[1,0,0]
    crossx = ny[0,0,0]*nz[1,0,0] - nz[0,0,0]*ny[1,0,0]
    sfac = 0.5*(s[0,0,0]+s[1,0,0])
    e += (1.0-dott*dott)-KK*dott*crossx*sfac
    # i - 1
    dott = nx[0,0,0]*nx[-1,0,0] + ny[0,0,0]*ny[-1,0,0] + nz[0,0,0]*nz[-1,0,0]
    crossx = ny[0,0,0]*nz[-1,0,0] - nz[0,0,0]*ny[-1,0,0]
    sfac = 0.5*(s[0,0,0]+s[-1,0,0])
    e += (1.0-dott*dott)+KK*dott*crossx*sfac
    # j + 1
    dott = nx[0,0,0]*nx[0,1,0] + ny[0,0,0]*ny[0,1,0] + nz[0,0,0]*nz[0,1,0]
    crossy = nz[0,0,0]*nx[0,1,0] - nx[0,0,0]*nz[0,1,0]
    sfac = 0.5*(s[0,0,0]+s[0,1,0])
    e += (1.0-dott*dott)-KK*dott*crossy*sfac
    # j - 1
    dott = nx[0,0,0]*nx[0,-1,0] + ny[0,0,0]*ny[0,-1,0] + nz[0,0,0]*nz[0,-1,0]
    crossy = nz[0,0,0]*nx[0,-1,0] - nx[0,0,0]*nz[0,-1,0]
    sfac = 0.5*(s[0,0,0]*s[0,-1,0])
    e += (1.0-dott*dott)+KK*dott*crossy*sfac
    # k + 1
    dott = nx[0,0,0]*nx[0,0,1] + ny[0,0,0]*ny[0,0,1] + nz[0,0,0]*nz[0,0,1]
    crossz = nx[0,0,0]*ny[0,0,1] - ny[0,0,0]*nx[0,0,1]
    sfac = 0.5*(s[0,0,0]*s[0,0,1])
    e += (1.0-dott*dott)-KK*dott*crossz*sfac
    # k - 1
    dott = nx[0,0,0]*nx[0,0,-1] + ny[0,0,0]*ny[0,0,-1] + nz[0,0,0]*nz[0,0,-1]
    crossz = nx[0,0,0]*ny[0,0,-1] - ny[0,0,0]*nx[0,0,-1]
    sfac = 0.5*(s[0,0,0]*s[0,0,-1])
    e += (1.0-dott*dott)+KK*dott*crossz*sfac
    return e
E = ekernel(nx,ny,nz,s)
print(E.shape)
E = E[1:-1,1:-1,1:-1]
print(E.shape)
print(E)

In [ ]:
def plot_quadrants(ax, array, fixed_coord, cmap):
    """For a given 3d *array* plot a plane with *fixed_coord*, using four quadrants."""
    nx, ny, nz = array.shape
    index = {
        'x': (nx // 2, slice(None), slice(None)),
        'y': (slice(None), ny // 2, slice(None)),
        'z': (slice(None), slice(None), nz // 2),
    }[fixed_coord]
    plane_data = array[index]

    n0, n1 = plane_data.shape
    quadrants = [
        plane_data[:n0 // 2, :n1 // 2],
        plane_data[:n0 // 2, n1 // 2:],
        plane_data[n0 // 2:, :n1 // 2],
        plane_data[n0 // 2:, n1 // 2:]
    ]

    min_val = array.min()
    max_val = array.max()

    cmap = plt.get_cmap(cmap)

    for i, quadrant in enumerate(quadrants):
        facecolors = cmap((quadrant - min_val) / (max_val - min_val))
        if fixed_coord == 'x':
            Y, Z = np.mgrid[0:ny // 2, 0:nz // 2]
            X = nx // 2 * np.ones_like(Y)
            Y_offset = (i // 2) * ny // 2
            Z_offset = (i % 2) * nz // 2
            ax.plot_surface(X, Y + Y_offset, Z + Z_offset, rstride=1, cstride=1,
                            facecolors=facecolors, shade=False)
        elif fixed_coord == 'y':
            X, Z = np.mgrid[0:nx // 2, 0:nz // 2]
            Y = ny // 2 * np.ones_like(X)
            X_offset = (i // 2) * nx // 2
            Z_offset = (i % 2) * nz // 2
            ax.plot_surface(X + X_offset, Y, Z + Z_offset, rstride=1, cstride=1,
                            facecolors=facecolors, shade=False)
        elif fixed_coord == 'z':
            X, Y = np.mgrid[0:nx // 2, 0:ny // 2]
            Z = nz // 2 * np.ones_like(X)
            X_offset = (i // 2) * nx // 2
            Y_offset = (i % 2) * ny // 2
            ax.plot_surface(X + X_offset, Y + Y_offset, Z, rstride=1, cstride=1,
                            facecolors=facecolors, shade=False)


def figure_3D_array_slices(array, cmap=None):
    """Plot a 3d array using three intersecting centered planes."""
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_box_aspect(array.shape)
    plot_quadrants(ax, array, 'x', cmap=cmap)
    plot_quadrants(ax, array, 'y', cmap=cmap)
    plot_quadrants(ax, array, 'z', cmap=cmap)
    return fig, ax


figure_3D_array_slices(E, cmap='viridis')
plt.show()

In [ ]:
#3d energy plot
fig,ax = plt.subplots(1,1,figsize=(6,6),dpi=125,subplot_kw={"projection": "3d"})
X,Y,Z = np.meshgrid(np.arange(E.shape[0]),np.arange(E.shape[1]),np.arange(E.shape[2]))
_ = ax.contourf(X[:,:,0],Y[:,:,0],E[:,:,0],zdir='z',offset=0)
_ = ax.contourf(X[0,:,:,],E[0,:,:],Z[0,:,:],zdir='y',offset=0)
C = ax.contourf(E[:,-1,:],Y[:,-1,:],Z[:,-1,:],zdir='x',offset=X.max())
# Set limits of the plot from coord limits
xmin, xmax = X.min(), X.max()
ymin, ymax = Y.min(), Y.max()
zmin, zmax = Z.min(), Z.max()
ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax], zlim=[zmin, zmax])

# Plot edges
edges_kw = dict(color='0.4', linewidth=1, zorder=1e3)
ax.plot([xmax, xmax], [ymin, ymax], 0, **edges_kw)
ax.plot([xmin, xmax], [ymin, ymin], 0, **edges_kw)
ax.plot([xmax, xmax], [ymin, ymin], [zmin, zmax], **edges_kw)

# Set zoom and angle view
ax.view_init(40, -30, 0)
ax.set_box_aspect(None, zoom=0.9)

# Colorbar
fig.colorbar(C, ax=ax, fraction=0.02, pad=0.1, label='Energy')
print(E.max(),E.min())
# Show Figure
plt.show()

In [ ]:
#3d energy plot
import plotly.graph_objects as go
from plotly.offline import iplot,init_notebook_mode
init_notebook_mode(connected=True)

E = ekernel(nx,ny,nz,s)
shape = E.shape
iso_value = 2.0

x,y,z = np.indices(shape)
fig = go.Figure(data = go.Isosurface(
                x = x.flatten(),
                y = y.flatten(),
                z = z.flatten(),
                value = E.flatten(),
                colorscale='Viridis',
                opacity=0.6,
                surface_count=1,
                isomin = iso_value,
                isomax = iso_value))
fig.update_layout(
    autosize=True,
    width=800,
    height=800,
    title = "3D Energy IsoSurface",
    scene = dict(
        xaxis_title = "X Axis",
        yaxis_title = "Y Axis",
        zaxis_title = "Z Axis"))
    #margin = dict(l=40,r=40,b=80,t=40,pad=4))
fig.show()